# CWL Execution - *CommandLineTool* Class 

Link to the documentation: https://eoap.github.io/quickwin/cwl-cli/water_bodies/

In this Notebook we will inspect the CWL file saved in `../cwl-cli/detect-water-body.cwl`

In [ ]:
cat ../cwl-cli/detect-water-body.cwl | yq e . -

Let's inspect a few elements of this CWL CommandLineTool

What CWL class is it?

In [ ]:
cat ../cwl-cli/detect-water-body.cwl | yq e .class -

What's its `id` ?

In [ ]:
cat ../cwl-cli/detect-water-body.cwl | yq e .id -

Inspect the base command and arguments:

In [ ]:
cat ../cwl-cli/detect-water-body.cwl | yq e .baseCommand -
cat ../cwl-cli/detect-water-body.cwl | yq e .arguments -

This is equivalent to: 

```yaml
baseCommand: ["python"]
arguments: ["-m", "app"]
```

Let's inspect the `inputs` element:

In [ ]:
cat ../cwl-cli/detect-water-body.cwl | yq e .inputs -

The `inputBinding` mapping provides a prefix to get to:

```
python -m app \
    --input-item <item input value> \
    --aoi <aoi input value> \
    --epsg <epsg input value> \
    --band <band input value [0]> \
    --band <band input value [1]> ...
```

Let's inspect the `outputs`.

Since the Python command line tool generates a STAC catalog, a STAC item and a geotiff, the glob expression will list all produced files in the folder the tool is run in.

In [ ]:
cat ../cwl-cli/detect-water-body.cwl | yq e .outputs -

Let's inspect the `requirements` element:

In [ ]:
cat ../cwl-cli/detect-water-body.cwl | yq e .requirements -

The `EnvVarRequirement` adds an environment variable, an equivalent of:

```
export PYTHON=/app
```

The `ResourceRequirement` sets the computing resources requests, in this case one CPU and 512 Mib of RAM.

The `DockerRequirement` tels what container image to use, in this case `localhost/detect-water-body:latest`

**Final note:** all CWL documents must declare the CWL version used: 



In [ ]:
cat ../cwl-cli/detect-water-body.cwl | yq e .cwlVersion -

Let's use a CWL runner to run this CWL document.

First create the `params.yaml` file:


In [ ]:
cat << EOF > params.yaml
item: https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2B_10TFK_20210713_0_L2A
aoi: "-121.399,39.834,-120.74,40.472"
epsg: "EPSG:4326"
band: 
- green
- nir
EOF

Then use the `cwltool` to run the CWL document against the parameters:

In [ ]:
cwltool ../cwl-cli/detect-water-body.cwl params.yaml > output.json

Inspect the `output.json` file keys:

In [ ]:
cat output.json | yq e '. | keys' - 

They're the same as: 

In [ ]:
cat ../cwl-cli/detect-water-body.cwl | yq e .outputs -

Now get the root folder of the results:

In [ ]:
cat output.json | yq e '.water-body.location' -

We know there's a catalog.json file there:


In [ ]:
stac describe $( cat output.json | yq e '.water-body.location' - | sed 's|file://||g' )/catalog.json